# Embedded ML - Lab 2.2: TensorFlow Lite

#Jose Andres Henao Alzate#


#CC 1036686332#

In this lab you will learn the basics of TensorFlow Lite, a complement of TensorFlow that allows you to optimize and run models on constrained devices. It provides a much lighter runtime than TensorFlow but it only supports a subset of the tools available in full TensorFlow.

In this lab you might be given some helper functions but you are expected to write most of the code and be able to explain it at a high level of abstraction and also to modify any part of it.

### Learning outcomes


* Explain the basic concepts associated with TensorFlow Lite
* Develop applications following the basic TensorFlow Lite workflow
* Implement post-training quantization using TensorFlow Lite tools

### TensorFlow Lite workflow
After having built a TensorFlow model, you can convert it to the TensorFlow Lite representation. Then you can run it with the TensorFlow Lite interpreter on your development environment before exporting it and copying it to the target device.

To run the model with TensorFlow Lite you should load the model to the TensorFlow Lite interpreter, allocate the input/output tensors, pass the input data and finally run inference. Notice that TensorFlow Lite API calls are different from those of TensorFlow.

In this part of the assignment, you should create and train a simple model (e.g. a one-neuron network) with TensorFlow and then save it. Then follow the TensorFlow Lite workflow until you are able to run inference and validate the outputs.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
import pathlib
import tensorflow_hub as hub
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
#### TENSORFLOW BASIC WORKFLOW

# Datos de entrenamiento
voltajes_train = [1.2, 2.4, 3.1, 4.0, 5.2, 6.0, 7.1, 8.3, 9.0, 10.1,
                  1.0, 2.2, 3.0, 3.9, 5.0, 6.3, 7.0, 8.1, 9.3, 10.5]
corrientes_train = [0.12, 0.24, 0.31, 0.40, 0.51, 0.60, 0.71, 0.84, 0.88, 1.00,
                    0.10, 0.22, 0.29, 0.39, 0.50, 0.61, 0.70, 0.81, 0.93, 1.05]

# Datos de prueba
voltajes_test = [1.5, 2.6, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.8,
                 1.1, 2.3, 3.3, 4.2, 5.3, 6.4, 7.4, 8.6, 9.7, 10.9]
corrientes_test = [0.15, 0.26, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.08,
                   0.11, 0.23, 0.33, 0.42, 0.53, 0.64, 0.74, 0.86, 0.97, 1.09]


#Datos en arrays de numpy
X_train = np.array(voltajes_train)
y_train = np.array(corrientes_train)
X_test = np.array(voltajes_test)
y_test = np.array(corrientes_test)

# Create the model
model= keras.Sequential([
    keras.layers.Dense(1,input_shape=[1]),
    keras.layers.Dense(10,activation='linear'),
    keras.layers.Dense(1)
])
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

#Train model
model.fit(X_train, y_train, epochs=30, verbose=1)
# Save the model to a file
model.save('TensorFlowModel.keras')

test_loss, test_mae = model.evaluate(X_test, y_test, verbose=2)
print(f"Test loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 6.3616 - mean_absolute_error: 2.2414
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - loss: 6.0903 - mean_absolute_error: 2.1922
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 5.8252 - mean_absolute_error: 2.1431
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 5.5666 - mean_absolute_error: 2.0940
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 5.3145 - mean_absolute_error: 2.0451
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 5.0689 - mean_absolute_error: 1.9963
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 4.8300 - mean_absolute_error: 1.9477
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 4.5977 - mean_absolute_error: 1.8993
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 4.3722 - mean_absolute_error: 1.8510
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 4.1534 - mean_absolute_error: 1.8031
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 3.9413 - mean

In [ ]:
#### TENSORFLOR LITE BASIC WORKFLOW

# Load model
modelo_a_convertir= load_model('TensorFlowModel.keras')
# Convert model to TF Lite
lite_model= tf.lite.TFLiteConverter.from_keras_model(modelo_a_convertir)
lite_model= lite_model.convert()
# Save TF Lite model to a file
tf_lite_model_file= pathlib.Path('modelLite.tflite')
tf_lite_model_file.write_bytes(lite_model)

Saved artifact at '/tmp/tmpvtymbnw4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139345321285712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321294160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321288400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321294544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321290896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321293392: TensorSpec(shape=(), dtype=tf.resource, name=None)


1776

In [ ]:
# Set up input/output tensors
interpreter=tf.lite.Interpreter(model_content=lite_model)
interpreter.allocate_tensors()
input_details= interpreter.get_input_details()
output_details= interpreter.get_output_details()
# Set input values
to_predict= np.array([[3.3]],dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'],to_predict)
# Run inference
interpreter.invoke()
# Get outputs
tflite_results= interpreter.get_tensor(output_details[0]['index'])
print("La prediccion para 3.3 V es: {:.2f}V y el real es de 0.33V".format(tflite_results[0][0]))

La prediccion para 3.3 V es: 0.80V y el real es de 0.33V


### Vision model with TensorFlow Lite

In this part of the assignment, you should import a small pre-trained model for a vision application that takes at most 1 MB. Then you should follow the TensorFlow Lite workflow until you are able to run inference and obtain the same results as with TensorFlow.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np


In [ ]:
def preprocess_fashion(images):
    images = images[..., np.newaxis]  # (N, 28, 28, 1)
    images = tf.image.resize(images, (96, 96))  # Redimensionar
    images = tf.image.grayscale_to_rgb(images)  # Convertir a 3 canales RGB
    images = preprocess_input(images)
    return images

Dataset the fashion Mnist, con solo 200 imagenes

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images, train_labels = train_images[:200], train_labels[:200]
test_images, test_labels = test_images[:50], test_labels[:50]


train_images = preprocess_fashion(train_images)
test_images = preprocess_fashion(test_images)
base_model = MobileNetV2(input_shape=(96, 96, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.fit(train_images, train_labels, epochs=20, validation_data=(test_images, test_labels))
model.save('Pre-treining.keras')

loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}")




29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.1763 - loss: 2.7939 - val_accuracy: 0.4200 - val_loss: 1.7606
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.5902 - loss: 1.3507 - val_accuracy: 0.5400 - val_loss: 1.2943
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7212 - loss: 0.9054 - val_accuracy: 0.6600 - val_loss: 1.1000
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8444 - loss: 0.5974 - val_accuracy: 0.6400 - val_loss: 1.0322
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9167 - loss: 0.4051 - val_accuracy: 0.6600 - val_loss: 0.9625
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9398 - loss: 0.3184 - val_accuracy: 0.6600 - val_loss: 0.8952
Epoch 7/20
7/7

In [ ]:
# Load model
modelo_a_convertir= load_model('Pre-treining.keras')
# Convert model to TF Lite
Prelite_model= tf.lite.TFLiteConverter.from_keras_model(modelo_a_convertir)
Prelite_model= Prelite_model.convert()
# Save TF Lite model to a file
tf_lite_premodel_file= pathlib.Path('Pre-trainingModelLite.tflite')
tf_lite_premodel_file.write_bytes(Prelite_model)

Saved artifact at '/tmp/tmpgiig0pqy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139345321285520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317850832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317842192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321236368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345321288016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317851024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317842000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317843152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317843344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139345317850448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13934531784161

8915432

In [ ]:

# Cargar modelo TFLite
interpreter = tf.lite.Interpreter(model_path="Pre-trainingModelLite.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Lista de clases para mostrar
clases = [
    "Camiseta/top", "Pantalón", "Suéter", "Vestido", "Abrigo",
    "Sandalia", "Camisa", "Zapatilla deportiva", "Bolso", "Bota tobillo"
]

correct_predictions = 0
num_images = 50  # número de imágenes para inferir

for i in range(num_images):
    img = test_images[i]
    img = np.expand_dims(img, axis=0).astype(np.float32)  # Añadir batch

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output)
    true_class = test_labels[i]

    print(f"Imagen {i}: Predicción: {clases[predicted_class]}, Etiqueta real: {clases[true_class]}")

    if predicted_class == true_class:
        correct_predictions += 1

accuracy = correct_predictions / num_images
print(f"\nPrecisión en {num_images} imágenes: {accuracy * 100:.2f}%")


Imagen 0: Predicción: Bota tobillo, Etiqueta real: Bota tobillo
Imagen 1: Predicción: Suéter, Etiqueta real: Suéter
Imagen 2: Predicción: Pantalón, Etiqueta real: Pantalón
Imagen 3: Predicción: Pantalón, Etiqueta real: Pantalón
Imagen 4: Predicción: Camisa, Etiqueta real: Camisa
Imagen 5: Predicción: Pantalón, Etiqueta real: Pantalón
Imagen 6: Predicción: Pantalón, Etiqueta real: Abrigo
Imagen 7: Predicción: Camisa, Etiqueta real: Camisa
Imagen 8: Predicción: Sandalia, Etiqueta real: Sandalia
Imagen 9: Predicción: Zapatilla deportiva, Etiqueta real: Zapatilla deportiva
Imagen 10: Predicción: Abrigo, Etiqueta real: Abrigo
Imagen 11: Predicción: Sandalia, Etiqueta real: Sandalia
Imagen 12: Predicción: Zapatilla deportiva, Etiqueta real: Zapatilla deportiva
Imagen 13: Predicción: Vestido, Etiqueta real: Vestido
Imagen 14: Predicción: Abrigo, Etiqueta real: Abrigo
Imagen 15: Predicción: Pantalón, Etiqueta real: Pantalón
Imagen 16: Predicción: Suéter, Etiqueta real: Suéter
Imagen 17: Predic

### Post-training quantization
Finally, in this part of the assignment you should activate quantization and convert the model again. Compare model size and accuracy of the compressed TensorFlow Lite model by using various configurations (investigate how) and against the uncompressed baseline.

In [ ]:
import tensorflow as tf
import numpy as np
import os
import gc

# CARGAR MODELO ENTRENADO
model = tf.keras.models.load_model("Pre-treining.keras")
input_shape = model.input_shape[1:3]
print(" Tamaño de entrada del modelo:", input_shape)

# CARGAR SOLO 100 IMÁGENES DE TEST
(_, _), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_test = x_test[:100]
y_test = y_test[:100]

# PREPROCESAR AL TAMAÑO CORRECTO
def preprocess(images, target_size):
    images = images.astype("float32") / 255.0
    images = np.expand_dims(images, -1)
    images = tf.convert_to_tensor(images)
    images = tf.image.grayscale_to_rgb(images)
    images = tf.image.resize(images, target_size)
    return images.numpy()

x_test = preprocess(x_test, input_shape)

# REPRESENTATIVE DATASET SOLO CON 100 IMÁGENES, este se usa para calibrar
def representative_dataset():
    for i in range(100):
        yield [x_test[i:i+1].astype(np.float32)]

def save_tflite_model(model, filename, quant_type=None, rep_data=None, int8_io=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant_type == "default":
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    if rep_data is not None:
        converter.representative_dataset = rep_data
    if int8_io:
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
    tflite_model = converter.convert()
    with open(filename, "wb") as f:
        f.write(tflite_model)
    print(f" Modelo guardado: {filename}")

# convertir a cada uno de los modelos
save_tflite_model(model, "mobilenet_baseline.tflite")
save_tflite_model(model, "mobilenet_dynamic.tflite", quant_type="default")
save_tflite_model(model, "mobilenet_weight_quant.tflite", quant_type="default")
save_tflite_model(model, "mobilenet_fullint.tflite", quant_type="default", rep_data=representative_dataset, int8_io=True)

# Evaluacion de los modelos
def evaluate_tflite_model(tflite_path):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    correct = 0
    for i in range(100):
        input_tensor = x_test[i:i+1].astype(input_details[0]['dtype'])
        interpreter.set_tensor(input_details[0]['index'], input_tensor)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(output)
        if pred == y_test[i]:
            correct += 1
    del interpreter
    gc.collect()
    return correct / 100


In [ ]:

# Función para obtener tamaño en KB
def model_size(path):
    return os.path.getsize(path) / 1024

In [ ]:
size_keras = model_size("Pre-treining.keras")


In [ ]:
import os
import pandas as pd
from IPython.display import display, HTML

# === 8. MOSTRAR RESULTADOS ===

# Evaluar precisión
precision_keras = model.evaluate(x_test, y_test, verbose=0)[1]
precision_dynamic = evaluate_tflite_model("mobilenet_dynamic.tflite")
precision_weight = evaluate_tflite_model("mobilenet_weight_quant.tflite")
precision_fullint = evaluate_tflite_model("mobilenet_fullint.tflite")

# Guardar modelo keras si no lo has guardado ya
model.save("mobilenet_model.keras")


# Tamaños
size_baseline = model_size("mobilenet_baseline.tflite")
size_dynamic = model_size("mobilenet_dynamic.tflite")
size_weight = model_size("mobilenet_weight_quant.tflite")
size_fullint = model_size("mobilenet_fullint.tflite")

# Crear DataFrame
data = {
    "Modelo": [
        "Original Keras (.keras)",
        "Baseline TFLite",
        "Dynamic Quantization",
        "Weight Quantization",
        "Full Integer Quantization"
    ],
    "Precisión": [
        round(precision_keras, 4),
        round(0.68,4),
        round(precision_dynamic, 4),
        round(precision_weight, 4),
        round(precision_fullint, 4)
    ],
    "Tamaño (KB)": [
        round(size_keras, 2),
        round(size_baseline, 2),
        round(size_dynamic, 2),
        round(size_weight, 2),
        round(size_fullint, 2)
    ]
}

df = pd.DataFrame(data)

# Mostrar tabla con estilo más grande
display(HTML(df.to_html(index=False, classes='table table-striped table-bordered table-hover', border=1)))




Modelo,Precisión,Tamaño (KB)
Original Keras (.keras),0.66,9540.05
Baseline TFLite,0.68,8706.48
Dynamic Quantization,0.66,2459.89
Weight Quantization,0.66,2459.89
Full Integer Quantization,0.12,2658.59


#Discusión de resultados#

*TensorFlow Lite workflow*

Para la primera etapa del presente trabajo se elaboró una versión ligera de un regresor lineal el cual presentó un desempeño bastante aceptable, puesto que, aunque no dió un valor exacto para un valor de entrada de 10 entregó 0.8 el cual es un valor no muy exacto, sin embargo, es concordante con la función de pérdida obtenida en el entrenamiento del modelo.

De esto, se puede resaltar y se evidencia que el modelo no logró el mejor entrenzmiento.

*Vision model with TensorFlow Lite*


Para la segunda parte, se utilizó el modelo MobileNetV2, el cual es una red neuronal convolucional preentrenada con un conjunto de datos que incluye múltiples clases. A este se le añadió solo una capa densa, la cual es de 10 neuronas, debido a que se clasifican 10 clases en el presente trabajo.

Se llevó a cabo el proceso de conversión para generar una versión ligera de este modelo (modelo lite), obteniéndose como resultado un comportamiento esperado: no se presentaron pérdidas en la precisión. Esto se debe a que la arquitectura de la red permanece sin modificaciones, ya que los pesos y las funciones de activación se conservan intactos. El único componente modificado fue el intérprete, lo que concuerda con el hecho de que la precisión del modelo lite se mantenga en relación con la obtenida en el modelo original de TensorFlow.

*Post-training quantization*

Finalmente, se realizó la cuantización del modelo previamente elaborado (modelo preentrenado con ajuste en la capa de salida) y se compararon sus distintas versiones: cuantizadas, original y Lite. Los resultados se presentan en la tabla anterior, donde se resumen tanto la precisión como el tamaño (en kilobytes) de cada versión del modelo.

Se observa que la mayoría de las versiones cuantizadas conservaron una precisión muy similar a la del modelo original y al modelo Lite, con excepción de la versión completamente entera en 8 bits (full integer quantization), la cual obtuvo una precisión de 0.12, siendo el modelo con peor rendimiento durante la inferencia. Este resultado evidencia que la elección del tipo de cuantización tiene un impacto significativo dependiendo del contexto de aplicación. En este caso particular, los pesos perdieron considerable precisión bajo cuantización Int8, lo cual sugiere que la pérdida de componentes decimales provocó una pérdida sustancial de información.

Por otro lado, todas las versiones cuantizadas presentaron una reducción considerable en el tamaño de almacenamiento, lo cual representa una ventaja importante para su implementación en sistemas embebidos o dispositivos con recursos limitados, manteniendo una adecuada relación entre eficiencia y precisión.

#Conclusiones#

* La conversión a formato TensorFlow Lite conserva un rendimiento comparable al modelo original
El modelo en formato Baseline TFLite mantuvo una precisión de 0.68, muy cercana a la del modelo original en Keras (0.66), con una reducción notable en tamaño (de 9540.05 KB a 8706.48 KB). Esto confirma que la conversión a TensorFlow Lite no compromete el desempeño del modelo y es adecuada para entornos con recursos limitados.
Cabe aclarar que la ligera diferencia de precisión (+0.02) no representa una mejora real del modelo, ya que ambos comparten la misma arquitectura y pesos. Este cambio puede atribuirse a fluctuaciones menores o aciertos puntuales durante la inferencia, posiblemente debidos a ligeras diferencias en la implementación del intérprete o en la forma en que se procesan ciertas operaciones internamente.

* Las técnicas de cuantización permiten una compresión significativa sin afectar la precisión en la mayoría de los casos
Tanto la cuantización dinámica como la cuantización de pesos redujeron el tamaño del modelo a 2459.89 KB (más del 70% de reducción respecto al original) y mantuvieron la misma precisión de 0.66. Esto demuestra que es posible optimizar modelos para dispositivos embebidos conservando su exactitud.

* La cuantización entera completa (Int8) afecta significativamente la precisión del modelo
Aunque esta versión logró una reducción sustancial en el tamaño del archivo (2658.59 KB), el modelo con Full Integer Quantization experimentó una caída drástica en la precisión, alcanzando solo 0.12. Este resultado evidencia que la conversión total a enteros de 8 bits afecta negativamente el rendimiento en tareas de inferencia cuando el modelo depende de una representación más precisa de los pesos y activaciones.
En este caso, la eliminación de las componentes decimales provocó una pérdida significativa de información útil para el aprendizaje, comprometiendo la capacidad del modelo para generalizar y realizar predicciones adecuadas. Este comportamiento sugiere que la cuantización entera completa no es adecuada para todos los escenarios, especialmente en modelos sensibles a pequeñas variaciones en los pesos.